# EEE3092F Assignment 2 - Julia Exercises_Section_2.5

#### PLAGIARISM STATEMENT

I certify that this assignment/report is my own work, based on my personal
study and/or research and that I have acknowledged all material and sources
used in its preparation, and that I
have not copied in part or whole or otherwise plagiarised the work of other
students and/or persons.

Name: Nkosinathi Ntuli (NTLNKO007)

Date: 15 April 2022

In [ ]:
using Pkg

In [ ]:
Pkg.add(["Plots", "PlotlyBase", "WAV", "StatsBase"]);

### Exercise 2.5.1 – Visualising Sampled Sinusoid

In [ ]:
using Plots

#plotly(); # comment out plotly and use gr to reduce the file size for submission
gr();

(a) 100x the Nyquist rate. 

In [ ]:
default(size=(800,400)); # Plot canvas size
default(label=""); # Turn off legends
default(ticks=:native); # 

function myplot(k)
    f0 = 10
    
    fnyquist = 2*f0 # Nyquist rate
    fs = k*fnyquist # sample rate
    Δt = 1/fs # time step
    t = 0:Δt:10/f0 # range 

    x = sin.(2π*f0*t) # Array containing function to be plotted 
    
    # Ploting using Plots + Plotly backend
    fig = plot(t, x,  
    title = "Lines joining the dots f₀ = $(f0)Hz fₛ = $(fs)Hz",
    xlabel = "t [s]",
    ylabel = "x(t)");
    display(fig);
    
    fig = plot(t, x, 
        seriestype = :scatter, 
        markersize=2, 
        title = "Scatter Plot showing the samples f₀ = $(f0)Hz fₛ = $(fs)Hz",
        xlabel = "t [s]");
    display(fig);
end

myplot(100);

(b) 10x the Nyquist rate.

In [ ]:
myplot(10);

(c) 2x the Nyquist rate.

In [ ]:
myplot(2);

(d) 1.1x the Nyquist rate.

In [ ]:
myplot(1.1);

(e) On the Nyquist rate.

In [ ]:
myplot(1);

(f) Below the Nyquist rate (try 0.7x, and 55x of the Nyquist rate). For these cases, calculate and display the frequency fs-f0

In [ ]:
myplot(0.7);
println("\nThe Plots with 0.7x the Nyquist Rate, fₛ - f₀ = $(0.7 * 2 * 10 - 10) Hz");

In [ ]:
myplot(55);
println("\nThe Plots with 55x the Nyquist Rate, fₛ - f₀ = $(55 * 2 * 10 - 10) Hz")

### Exercise 2.5.2 – DFT / FFT introduction

In [ ]:
using FFTW

In [ ]:
function dft(x)
    N=length(x)
    X = zeros(N)+im*zeros(N) # Complex array of 0+0im; Another method is X=zeros(ComplexF64,N)
    for k=1:N
        for n=1:N
            X[k] = X[k] + x[n]*exp(-im*2*pi*(k-1)*(n-1)/N)
        end
    end
    return X
end;

a) Insert this dft(x) function into Julia and compare it to the fft( ) function.

In [ ]:
x = [0,1,1,0,0,0,0,0]
@show fft(x);
@show dft(x);

b) Plot the magnitude and phase via

In [ ]:
X = fft(x);
using Plots
display(scatter(abs.(X), markersize=3, ylabel = "abs X[k]"))
display(scatter(angle.(X), markersize=3, ylabel = "angle X[k]"))

c) Try the ifft function.

In [ ]:
@show invX = ifft(X);
@show real(ifft(X));

d) Compare the speed of the dft() and fft() functions by timing doing some timing tests for different sizes of N. The fft() algorithm is fast (order NlogN) compared to the direct double for-loop DFT implementation (order N^2). Timing can be done using either the “@time” macro (which prints how long a function takes to evaluate, as well as memory used), or the “@elapsed” macro which returns the time taken so that it can be stored in a variable

In [ ]:
function timing_test(n)
    x = randn(n);
    t_dft = @elapsed dft(x);
    t_fft = @elapsed fft(x);
    println("dft of length $(n) took $(t_dft) seconds")
    println("fft of length $(n) took $(t_fft) seconds")
end;

In [ ]:
N = 1024
timing_test(N);

Trying with size N=2^8 = 256, N=2^10 = 1024, N=2^11 = 2028, N=2^12 = 4096. 

In [ ]:
N_sizes = [256, 1024, 2028, 4096]
for i in N_sizes
    timing_test(i)
end

Questions:

(1) What is the largest power-of-2 size fft() that one can compute within 1 second (on your platform)? What is the largest dft() size?

In [ ]:
p = 12
t0 = @elapsed fft(randn(2^p))

while t0 < 1
    p = p + 1
    t0 = @elapsed fft(randn(2^p))
end

p1 = 12
t1 = @elapsed dft(randn(2^p1))

while t1 < 1
    p1 = p1 + 1
    t1 = @elapsed dft(randn(2^p1))
end


println("The largest for fft() is : 2^$(p1) = $(2^p1)")
println("The largest for dft() is : 2^$(p) = $(2^p)")

(2) Is the fft() really faster for powers of two? Time the fft for N=2^15=32768 and for N=32767. Which is faster?

In [ ]:
timing_test(2^15)

In [ ]:
timing_test(32767)

Yes, the fft() function is faster for powers of 2. The fft() function is faster than the dft() function.

### Exercise 2.5.3 – FFT of a sine wave

a) perform a spectral analysis on the signal _v(t) = 4 cos (20πt) + 2 cos (30πt)_. Show plots of the time domain, and the frequency domain.

In [ ]:
Δt = 0.001 # time step
t = 0:Δt:1; # range

vt = 4*cos.(20*π*t) + 2*cos.(30*π*t); # Create an array holding the sinusoid values


fig = plot(t, vt, 
    title = "v(t) = cos(20πt) + 2cos(30πt)", 
    markershape = :circle, 
    markerstrokecolor=:blue, 
    markersize = 2, 
    xlabel = "t [s]", 
    ylabel = "v(t)");

display(fig); # The time domain plot sampled signal

In [ ]:
#using FFTW   # Import Fourier library

# The frequency axis in Hz
N = length(t);
Δf = 1/(N*Δt)  # spacing in frequency domain

# case w/o fftshift
# create array of freq values stored in f_axis. First element maps to 0 Hz
f_axis = (0:N-1)*Δf;  

V = fft(vt)

# note: not normalised, to normalise multiply by (2/N)

fig = plot(f_axis, abs.(V), 
    markershape = :circle, 
    markersize = 3, 
    markercolor=:lightblue, 
    markerstrokecolor=:blue, 
    title = "The magnitude of the FFT(v)",
    ylabel = "abs V(k)",
    xlabel = "f [Hz]");

display(fig);

# for fftshift case
# create array of freq values stored in f_axis. 
if mod(N,2)==0    # case N even
    f_axis = (-N/2:N/2-1)*Δf;    
else   # case N odd
    f_axis = (-(N-1)/2 : (N-1)/2)*Δf; 
end

# swap around halves of array so that plot show zero Hz in middle
fig = plot(f_axis, fftshift(abs.(V)), 
    markershape = :circle, 
    markersize = 3, 
    markercolor=:lightblue, 
    markerstrokecolor=:blue, 
    title = "The fftshift of the magnitude of the FFT(v)",
    ylabel = "",
    xlabel = "f [Hz]");

display(fig);

b) Try zero padding in the time domain, i.e. extend the length of the time array by adding additional zeros. This will give a finer sample spacing in the frequency domain.

In [ ]:
y = zeros(16*N) # Make array 16x longer.
y[1:N] = t; # Copy x into first N samples. The rest contains zeros.
Y = fft(y);

fig = plot(abs.(Y), title = "The magnitude of the FFT(y)");
display(fig);

fig = plot(abs.(Y[1:200]), title = "The magnitude of the FFT(y) [zoomed in first 200 samples]");
display(fig);

N_new = length(y)
Δf_new = 1/(N_new*Δt)  # new spacing in frequency domain
#create array of freq values stored in f_axis. First element maps to 0Hz
if mod(N_new,2)==0    # case N even
    f_axis_new = (-N_new/2:N_new/2-1)*Δf_new;    
else   # case N odd
    f_axis_new = (-(N_new-1)/2 : (N_new-1)/2)*Δf_new; 
end

fig = plot(f_axis_new, fftshift(abs.(Y)), 
    title = "The fftshift of the magnitude of the FFT(y)",
    xlabel = "f [Hz]");
display(fig);

Question: What do you now clearly see at the locations of the two sinusoidal frequencies?

The Sa() functions

### Exercise 2.5.4 – Effect of ADC quantization¶

Simulate a sinusoid voltage (as in the previous exercise). _v = cos.(2pif0*t)_ The signal lies in the range: _Amin = -0.5_ to _Amax = 0.5_ Quantize the signal into 2^Nbits levels when Nbits is the number of bits of an ADC.

In [ ]:
f0 = 10 # fundamental frequency
Δt = 0.001 
t = -0.5:Δt:0.5; # range
A = 1 # amplitude in V

vt2 = A*cos.(2*pi*f0*t); #sinusoid voltage 


fig = plot(t, vt2, 
    markershape = :circle, 
    markersize = 2, 
    markercolor=:lightblue, 
    markerstrokecolor=:blue, 
    title = "v = cos(2πf₀t)",
    xlabel = "t [s]", 
    ylabel = "v(t)");

display(fig);

In [ ]:
Nbits = 4 # Try 2, then 3, then 4 etc.
Nlevels = 2^Nbits
Amax = 1+0.00001 # Add a small amount to prevent problem at extreme
Amin = -1-0.00001
v_quantized = (round.( (vt2 .- Amin) ./(Amax-Amin)*Nlevels .- 0.5) .+0.5) / Nlevels*(Amax-Amin) .+ Amin;

# Display spectrum of the quantized signal in the time domain
display(
    plot(t, v_quantized, 
        title = "Quantized sinusoid vq(t) Nlevels = $(Nlevels)", 
        xlabel = "t [s]", 
        ylabel = "vq(t)",
        markershape = :circle, 
        markersize = 2, 
        markercolor=:lightblue, 
        markerstrokecolor=:blue)
    );



# Display spectrum of the unquantized signal
display( 
    plot(abs.(fft(vt2)),
        title = "The magnitude of the FFT(v)",
        markershape = :circle, 
        markersize = 2,
        markercolor=:lightblue, 
        markerstrokecolor=:blue));

# Display spectrum of the quantized signal
display( 
    plot(abs.(fft(v_quantized)), 
        title = "The magnitude of the FFT(vq(t))",
        markershape = :circle, 
        markersize = 2, 
        markercolor=:lightblue, 
        markerstrokecolor=:blue)
    );

display(
    plot(20*log10.(abs.(v_quantized)), 
        title = "The magnitude of the FFT(vq(t)) [dBv scale]",
        ylabel = "dB",
        markershape = :circle, 
        markersize = 1,
        markerstrokecolor=:blue);
    );

Having implemented this, inspect both the unquantized and quantized signal in the
time and frequency domains. Increase your sample rate to at least 10 times the
Nyquist (or higher) to see the effect.?



In [ ]:
fnyquist = 2*f0 # Nyquist rate

k = 100 # sample rate 

fs = k*fnyquist # sample rate
Δt = 1/fs # time step
t = 0:Δt:10/f0 # range 

vt2 = A*cos.(2*pi*f0*t)
v_quantized = (round.( (vt2 .- Amin) ./(Amax-Amin)*Nlevels .- 0.5) .+0.5) / Nlevels*(Amax-Amin) .+ Amin;

display(
    plot(t, vt2, title = "v = cos(2πf₀t)",
        xlabel = "t [s]", 
        ylabel = "v(t)",
        markershape = :circle, 
        markersize = 2, 
        markercolor=:lightblue, 
        markerstrokecolor=:blue
        )
    );
    
display(
    plot(t, v_quantized,
        title = "Quantized sinusoid vq(t) Nlevels = $(Nlevels)",
        xlabel = "t [s]", 
        ylabel = "vq(t)", 
        markershape = :circle, 
        markersize = 2, 
        markercolor=:lightblue, 
        markerstrokecolor=:blue));
        
# Display spectrum of the unquantized signal
display( 
    plot(abs.(fft(vt2)),
        title = "The magnitude of the FFT(v)",
        markershape = :circle, 
        markersize = 2,
        markercolor=:lightblue, 
        markerstrokecolor=:blue));

# Display spectrum of the quantized signal
display( 
    plot(abs.(fft(v_quantized)), 
        title = "The magnitude of the FFT(vq(t))",
        markershape = :circle, 
        markersize = 2, 
        markercolor=:lightblue, 
        markerstrokecolor=:blue)
    );

Question: What effect does quantization have in the frequency domain? 

As the Nyquist rate increase the quantization level become more finer, more points are represented

### Exercise 2.5.5 – Simulating bandlimited noise

a) Try the following:


In [ ]:
fs = 1000 # sample rate in Hz
Δt=1/fs; # sample spacing in s
N = 100 # Choose an even number (makes like easier later)
t = range(0, step=Δt, length=N) # Define time axis
σ = 1
x = σ * randn(N); # Create the random samples with std dev of σ.

fig = histogram(x,bins=10) # inspect histogram
display(fig);

fig = plot(t,x, 
    seriestype = :scatter, 
    markersize = 2,
    markerstrokecolor=:blue,
    title = "Sampled noise, bandwidth B=fs/2"); # inspect sampled time domain
display(fig);

X = fft(x);
fig = plot(abs.(X)); # inspect DFT frequency domain
display(fig);

b) Zero-pad in frequency domain to get finer sample spacing in time domain (read lecture notes). Run the code below to inspect the noise waveform resampled at 10x finer sample spacing.

In [ ]:
pad_factor=10
Ny = pad_factor*N;
Y = zeros(Ny)+im*zeros(Ny) # Create a complex array of zeros
k_mid = Int(N/2)
Y[1:k_mid]=X[1:k_mid]; # Insert the first half of X
Y[Ny-k_mid+1:Ny]=X[k_mid+1:N]; # Insert the 2nd half of X at the end

fig = plot(abs.(Y)); # inspect padded array
display(fig)

y = ifft(Y); # Go back to time domain
y = real(y); # discard the very tiny imaginary components
Ny = length(y)
t_new = range(0, step=Δt/pad_factor, length=Ny) # Define time axis

fig = plot(t_new,y,
    markershape = :circle, 
    markersize = 2, 
    markerstrokecolor=:blue);

display(fig)
fig = plot(t_new[1:300],y[1:300],
    markershape = :circle, 
    markersize = 2, 
    markerstrokecolor=:blue); # Plot just first 300 samples
display(fig)

c) Now try bandlimiting the noise using an ideal LPF. First create and display the filter. Then apply to the noise waveform in the frequency domain. Here the bandwidth B=100Hz. The original bandwidth was fs/2=1000/2=500Hz.

In [ ]:
# Create a Filter transfer function
Δω = 2*pi/(N*Δt) # Sample spacing in freq domain in rad/s
ω = 0:Δω:(N-1)*Δω
f = ω/(2*π)
B = 100 # filter bandwidth in Hz
# In the sampled frequency domain. add a rect centred on zero to one centred at the next repeat
# i.e. centred on 0 rad/s an on 2pi/Δt rad/s.
rect(t) = (abs.(t).<=0.5)*1.0;
H0 = rect(ω/(4*π*B)) + rect( (ω .- 2*π/Δt)/(4*π*B) );

fig = plot(f,H0,
    markershape = :circle, 
    markersize = 2, 
    markerstrokecolor=:blue,
    title = "Ideal LPF",
    xlabel = "Frequency in Hz"); # Note, H in this case is purely real.
display(fig)

In [ ]:
#Apply filter to noise:
X_filtered = X.*H0;

fig = plot(abs.(X_filtered))
display(fig)
x_filtered = ifft(X_filtered)
x_filtered = real(x_filtered)

fig = plot(t,x_filtered, 
    title = "Lowpass Filtered noise, bandwidth B=$(B)",
    xlabel = "time t [s]");
display(fig)

In [ ]:
using StatsBase;

How do the waveforms differ?

The lowpass filtered noise is more smooth

Calculate the standard deviations of each as a measure of spread

In [ ]:
@show std(x)
@show std(x_filtered);

### Exercise 2.5.6 – Discrete fast convolution

In [ ]:
rect(t) = (abs.(t).<=0.5)*1.0;

dt = 0.001
tmax = 8
T = 2

t=0:dt:tmax

x =  rect( (t.-T)/T )
y = rect( (t.-2T)/(2T) ) 
z = ifft(fft(x).*fft(y)); 

fig = plot(x)
fig = plot!(y)
display(fig);

In [ ]:
display(plot(real(z)));
display(plot(fftshift(z)));

### Exercise 2.5.7 – Spectral Analysis of Music

In [ ]:
using WAV

a) Write julia code to
- load an audio “wav” file
- extract a specified portion [t1,t2]
- display in time domain
- display in frequency domain

In [ ]:
function music_analyse(filename)
    data, fs = wavread(filename) # Read stereo wave file

    # wavread() returns 2D data array (matrix) and also the sample rate fs.
    # If stereo, the 1st column holds one channel and the 2nd column holds the other.

    channel1 = data[:,1] # extract 1st column (left channel I think)
    # channel2 = data[:,2] # if stereo, then there is a 2nd channel in the 2nd column.

    dt = 1/fs # calculate sample spacing from sample rate
    t_axis_all = (0:length(channel1)-1)*dt # time axis for plotting

    # Extracting a portion of sound from time t1 to t2 seconds
    t1 =0; t2=10; # specify start time t1 and end time t2
    i1 = Int(round(t1/dt))+1 # calculate index of sample closest to t1
    i2 = Int(round(t2/dt))+1 # calculate index of sample closest to t2

    y = channel1[i1:i2] # extract relevant portion from channel1

    N = length(y) # Get size of array

    t=(0:N-1)*dt; # Define a time axis for the extracted portion

    display(plot(t,y, title = "Time domain plot", xlabel="Time [seconds]")) # Plot array vs time
    
    # Next calculate the fft(y)
    # display abs.() of fft. Label frequenct axes in Hz etc.
    # For visualisation of a relevant portion use plot(array[index1:index2])
    #Label the frequency axis in Hz
    
    Δf = 1/(N*dt)  # spacing in frequency domain

    #create array of freq values stored in f_axis. First element maps to 0Hz
    f_axis = (0:N-1)*Δf;    

    Y = fft(y);
    display(plot(f_axis, abs.(Y), title = "Frequency domain plot",xlabel = "Frequency [Hz]"));
end;

Apply your code to three examples:Apply your code to three examples:


(1) Piano music: Beethoven - Für Elise (first 10 seconds)

Questions: Inspect the FFT. You should see the distinct piano notes in the spectrum. Determine the frequencies of the
highest three peaks. Determine which piano notes do these frequencies correspond to?
What are the frequencies of the highest and lowest notes played?

In [ ]:
music_analyse("../data/audio_files/Fur Elise.wav");


|Frequency (Hz)| Piano Notes|
|--------------|------------|
|     332      |     E4     |
|     498      |     B4     |
|     665      |     E5     |

(2) Master KG - Jerusalema [Feat. Nomcebo] (first 10 seconds)
Questions: Inspect the time domain and determine the period of the drum beat in seconds.
How does the (dominant) drum beat appear in the frequency spectrum? Describe its features.
How can you determine the fundamental frequency of the beat from inspection of the frequency spectrum?

In [ ]:
music_analyse("../data/audio_files/Master KG - Jerusalema [Feat. Nomcebo].wav");

T = 0.3 seconds

(3) A short recording of your own voice singing the chorus of Jerusalema (a capella).
https://lyricstranslate.com/en/jerusalema-jerusalem.html-0
Questions: Inspect the time domain and frequency spectrum. Describe significant features in time and frequency domains.
 How does the spectrum of your recording compare to (2)?

¯\\_(ツ)_/¯ Sorry, I can't sing that well :(